In [1]:
%matplotlib inline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
df=pd.DataFrame()
for u in range(1,4):
    u='%02d'%u
    l=pd.read_html(f"https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466920&stname=%25E8%2587%25BA%25E5%258C%2597&datepicker=2019-{u}#",
                   encoding = "utf-8")
    if '...' != pd.DataFrame(l[1]).iloc[0,1]:
        df=pd.concat([df,pd.DataFrame(l[1])],axis=0)    

In [ ]:
for i in range(len(df.columns)):
    if '氣溫(℃)' in df.columns[i]:
        t=i
    elif '相對溼度(%)' in df.columns[i]:
        rw=i
    elif '降水時數(hr)' in df.columns[i]:
        wq=i
    elif 'StnPresMaxTime' in df.columns[i]:
        tm=i

In [ ]:
df2=df.iloc[:,[tm,t,rw,wq]]

In [ ]:
df2.columns=['date','氣溫','相對溼度','降水時數']

In [ ]:
df2.index=range(len(df2.index))

In [ ]:
df2['溫差']=np.nan
for i in range(1,len(df2.index)-1):
    df2['溫差'][i]=float(df2['氣溫'][i])-float(df2['氣溫'][i-1])

In [ ]:
df2.index=pd.to_datetime(df2.loc[:,'date']).dt.date

In [ ]:
df2.drop('date',axis=1,inplace=True)

In [ ]:
import pandas_datareader.data as web

In [ ]:
dg = web.DataReader("^TWII","yahoo",start="2019-1-1",end="2019-3-31")

In [ ]:
dg['漲跌']=np.nan
for i in range(1,len(dg.index)-1):
    dg['漲跌'][i]=dg.Close[i]-dg.Close[i-1]

In [ ]:
dg['震幅']=dg.High-dg.Low

In [ ]:
dfg=df2.add(dg,fill_value=0)
dfg.dropna(inplace=True)

In [ ]:
dfg.corr()

In [ ]:
dfg.溫差.plot()

In [ ]:
dfg.漲跌.plot()

In [ ]:
dfg=dfg[['溫差','漲跌']]

In [ ]:
x=np.array(dfg.溫差)
y=np.array(dfg.漲跌)

In [ ]:
plt.scatter(x,y)

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=87)

x_train=x_train.reshape(len(x_train),1)

x_test.shape=(len(x_test),1)

from sklearn.linear_model import LinearRegression

regr=LinearRegression()

regr.fit(x_train,y_train)

In [ ]:
plt.scatter(x_train,y_train)
plt.scatter(x_test,y_test)
plt.plot(x_train,regr.predict(x_train),c='r')